In [9]:
from pymoo.core.problem import ElementwiseProblem
from pymoo.core.repair import Repair
from pymoo.optimize import minimize
from pymoo.core.crossover import Crossover
from pymoo.operators.crossover.binx import BX
from pymoo.core.mutation import Mutation
from pymoo.core.sampling import Sampling
from pymoo.core.population import Population
from pymoo.core.evaluator import Evaluator
from pymoo.util.ref_dirs import get_reference_directions
from pymoo.visualization.scatter import Scatter
from Plugins import pre_processing
from Plugins import ArchievingStrategies
from Plugins import my_plotting
from Plugins import pymoo_extras
from Plugins import DS
import numpy as np 
import pandas as pd

class Portfolio_Problem(ElementwiseProblem):
    def __init__(self,N, profits, risk,esg, **kwargs):
        self.profits = profits
        self.risk = risk
        self.esg = esg
        super().__init__(n_var=N,
                         n_obj=2,
                         xl=0.0,
                         xu=1.0,
                         **kwargs)

    def _evaluate(self, x, out, *args, **kwargs):
        #NORMALIZAR X antes de empezar
        s = np.sum(x)
        x = x/s
        #Función de evaluacion
        exp_return  =   self.profits@x
        exp_risk    =   np.sqrt(x.T@self.risk@x)
        exp_esg     =   self.esg@x
        sharpe      =   exp_return / exp_risk
        out['F']    =   [exp_risk, -exp_return]
        out['ESG']  =   exp_esg
        out['Sharpe'] = sharpe
        
class Portfolio_Repair(Repair):
    def _do(self, problem, X, **kwargs):
        X[X < 1e-3] = 0
        return X / X.sum(axis=1, keepdims=True)

In [38]:
class MySampling(Sampling):
    def _do(self, problem, n_samples, **kwargs):
        #100 individuos (a lo más 2000)
        #problem.n_var la cantidad de activos. 
        X = get_reference_directions("energy", problem.n_var, n_samples, seed=1)
        return X 
    
def point_on_triangle(pt1, pt2, pt3):
    s, t = sorted([np.random.random(), np.random.random()])
    return s*pt1 + (t-s)*pt2 + (1-t)*pt3

def SPX(x0, x1, x2, epsilon): 
    CoM =  (1/3)*(x0+x1+x2)
    V1=(x0-CoM)*(1+epsilon)
    V2=(x1-CoM)*(1+epsilon)
    V3=(x2-CoM)*(1+epsilon)   
    offspring = point_on_triangle(V1, V2, V3)
    return offspring
    
class MyCrossover(Crossover):
    def __init__(self):
        # define the crossover: number of parents and number of offsprings
        super().__init__(3, 1)

    def _do(self, problem, X, **kwargs):
        # The input of has the following shape (n_parents, n_matings, n_var)
        n_parents, n_matings, n_var = X.shape
        # The output owith the shape (n_offsprings, n_matings, n_var)
        # Because there the number of parents and offsprings are equal it keeps the shape of X
        _X = np.full((self.n_offsprings, n_matings, problem.n_var), 0)
        for k in range(n_matings): 
            x0, x1, x2 = X[0, k], X[1, k], X[2, k] 
            _X[0, k] = SPX(x0, x1, x2, epsilon=0.05) 
        return _X
    
class MyMutation(Mutation):
    def __init__(self):
        super().__init__()

    def _do(self, problem, X, **kwargs):

        # for each individual
        for i in range(len(X)):

            r = np.random.random()

            # with a probabilty of 40% - change the order of characters
            if r < 0.4:
                perm = np.random.permutation(problem.n_characters)
                X[i, 0] = "".join(np.array([e for e in X[i, 0]])[perm])

            # also with a probabilty of 40% - change a character randomly
            elif r < 0.8:
                prob = 1 / problem.n_characters
                mut = [c if np.random.random() > prob
                       else np.random.choice(problem.ALPHABET) for c in X[i, 0]]
                X[i, 0] = "".join(mut)

        return X

In [11]:
assets = pd.read_csv( 'Indices/tickers_dowjones.csv', index_col=0)['0'].tolist()
returns= pd.read_csv( 'data/dowjones_returns.csv', index_col=0)
assets_info=pd.read_csv( 'data/dowjones_assets_info.csv', index_col=0)
best_assets=pd.read_csv( 'data/dowjones_best_assets.csv', index_col=0)

In [24]:
PROFITS, RISK, ESG_SCORES = pre_processing.get_final_assets(returns[best_assets.index], assets_info.loc[best_assets.index])
portfolio_problem = pymoo_extras.Portfolio_Problem(len(PROFITS), PROFITS, RISK, ESG_SCORES)
from finquant import efficient_frontier
ef = efficient_frontier.EfficientFrontier(pd.Series(PROFITS), pd.DataFrame(RISK), freq=252)
ef_R = ef.efficient_frontier()

In [40]:
from pymoo.termination import get_termination
from pymoo.algorithms.moo.nsga2 import NSGA2
termination = get_termination("n_gen", 300)
nsgaii = NSGA2(pop_size=40,
               sampling=MySampling(), 
               crossover=MyCrossover(),
               repair=pymoo_extras.Portfolio_Repair())

In [31]:
eps = np.array([0.01,0.01])
colors = ['rgb(141,160,203)', 'rgb(27,158, 119)']

In [41]:
X_nsgaii, F_nsgaii, ESG_nsgaii =pymoo_extras.get_weights_with_pymoo(portfolio_problem, nsgaii, termination)
FA_nsgaii =  pymoo_extras.annualised_portfolio_quantities(F_nsgaii)
FA_nsgaii_best = ArchievingStrategies.get_best_opt(FA_nsgaii, 1e-6)
FA_nsgaii_best_eps = ArchievingStrategies.get_best_opt_eps(FA_nsgaii, 1e-6, eps)
print('Total', FA_nsgaii.shape)
print('Mejores', FA_nsgaii_best.shape)
print('Eps-Mejores', FA_nsgaii_best_eps.shape)

C:\Users\yeudi\AppData\Local\Temp\ipykernel_15824\2800248799.py:33: RuntimeWarning:

invalid value encountered in cast

12000it [00:10, 1108.23it/s]
12000it [00:13, 918.79it/s]

Total (12000, 2)
Mejores (11960, 2)
Eps-Mejores (11977, 2)


In [42]:
frames = [FA_nsgaii_best_eps, FA_nsgaii_best]
labels = ['PQ-eps', 'PQ']
fig=my_plotting.plotting_samples_plotly(ef_R, frames, labels, colors)
fig.show()